In [ ]:
from pathlib import Path
from utils import analyze_images

In [ ]:
# Define the dataset that you want to analyze below ("microglia" or "astrocyte")
dataset = "astrocyte"

if dataset == "microglia":
    directory_path = Path("./microglia_data/")
    print(f"The following dataset will be analyzed: {str(directory_path)}")
    
elif dataset == "astrocyte":
    directory_path = Path("./astrocyte_data/")
    print(f"The following dataset will be analyzed: {str(directory_path)}")
else:
    print("Please define an existing dataset")

In [ ]:
# Create an empty list to store all image filepaths within the dataset directory
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.lsm"):
    images.append(str(file_path))
    
images

In [ ]:
# Define analysis parameters

# Diameter in pixels of the nuclei present in your image, helps Cellpose to adjust nuclei mask predictions

# (cellpdia) in the results.csv file 
cellpose_nuclei_diameter = 30

# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)

# (sigma) in the results.csv file
gaussian_sigma = 1

# Defines how much the Cellpose detected nuclei are dilated in pixels to make sure the foci damage sit inside 

# (dilrad) in the results.csv file
dilation_radius_nuclei = 4

# Anything above this value is considered glia+ signal, it works well in scenarios where the glia staining intensities and background levels
# are uniform across samples, otherwise it causes issues of mislabeling (a high background signal will be considered as glia+)

# (gliathr) in the results.csv file
glia_channel_threshold = 20

# If set to True it overrides the thresholding method to detect glia+ signal and instead using APOC semantic classifier
# It has the advantage of detecting true glia+ signal across varying intensities but it is not as precise as thresholding and might introduce
# noise into your analysis results by segmenting suboptimal stainings.

# If True (glia_sem_seg_v) appears in the results.csv file instead of gliathr
glia_segmenter = False

# Selects the glia_segmenter version (so far I have only trained 1)

# (glia_sem_seg_v) in the results.csv file 
glia_segmenter_version = 1

# Sets the amount of erosion that is applied to areas where the glia+ signal colocalizes with nuclear signal
# This erosion is needed to remove small artifacts where a glial protrusion sits on top of a nuclei but does not correspond to a glial cell
# The higher the value, the stricter the conditions to consider a nuclei as glia+. Too high values will result in the complete absence of glia+ nuclei
# Initial analysis versions was 5, 6 removes most artifacts.

# (gliaero) in the results.csv file 
glia_nuclei_colocalization_erosion = 6 

# Selects the version of APOC object segmenter for detection of DNA damage foci
# 1 works well for fresh slides but not old stainings, 2 and 3 generalize better over fresh and old stainings.
# 3 is skewed towards detection of foci in suboptimal stains so it introduces some noise in the results. 1 is the strictest and 3 the loosest.

# (dnad_obj_seg_v) in the results.csv file
dna_damage_segmenter_version = 2

# This step allows you to fine tune the size of what is considered a spot and what is not, by increasing the dna_damage_erosion parameter you will consider only the bigger spots and discard the small ones.

# (dnadero) in the results.csv file
dna_damage_erosion = 2 


In [ ]:
analyze_images(images, cellpose_nuclei_diameter, gaussian_sigma, dilation_radius_nuclei, glia_channel_threshold, dna_damage_segmenter_version, glia_segmenter, glia_segmenter_version, glia_nuclei_colocalization_erosion, dna_damage_erosion, dataset)